# Introduction
I will add new information to TFIDF+NN model(https://www.kaggle.com/ryches/tfidf-benchmark ).<br>
TFIDF can create features based on actual vocabulary, but it can't handle well when there is another word of close meaning.<br>
Therefore, I thought that adding SWEM(https://arxiv.org/abs/1805.09843) using learned word2vec as a feature value would increase the score.

In [1]:
import numpy as np
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import gensim
from nltk.corpus import brown
import random
from sklearn.model_selection import KFold
import lightgbm as lgb
import gc
from keras.callbacks.callbacks import EarlyStopping
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks.callbacks import EarlyStopping
from scipy.stats import spearmanr
from nltk.corpus import wordnet as wn
import tqdm
from sklearn.model_selection import StratifiedKFold


Using TensorFlow backend.


In [146]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Flatten, Dense

In [2]:
train = pd.read_csv("../input/google-quest-challenge/train.csv")
test = pd.read_csv("../input/google-quest-challenge/test.csv")

In [3]:
sample_sub = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")

In [50]:
train.shape

(6079, 41)

In [4]:
target_cols = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']

In [122]:
len(train.columns)

41

In [123]:
len(target_cols)

30

# fearure engineering

In [5]:
def simple_prepro(s):
    return [w for w in s.replace("\n"," ").replace(","," , ").replace("("," ( ").replace(")"," ) ").
            replace("."," . ").replace("?"," ? ").replace(":"," : ").replace("n't"," not").
            replace("'ve"," have").replace("'re"," are").replace("'s"," is").split(" ") if w != ""]

In [6]:
def simple_prepro_tfidf(s):
    return " ".join([w for w in s.lower().replace("\n"," ").replace(","," , ").replace("("," ( ").replace(")"," ) ").
            replace("."," . ").replace("?"," ? ").replace(":"," : ").replace("n't"," not").
            replace("'ve"," have").replace("'re"," are").replace("'s"," is").split(" ") if w != ""])

This is basic preprocessing. This time, symbols and words are attached, so they are separated here.

In [75]:
qt_max = max([len(simple_prepro(l)) for l in list(train["question_title"].values)])
qb_max = max([len(simple_prepro(l))  for l in list(train["question_body"].values)])
an_max = max([len(simple_prepro(l))  for l in list(train["answer"].values)])
print("max lenght of question_title is",qt_max)
print("max lenght of question_body is",qb_max)
print("max lenght of question_answer is",an_max)

max lenght of question_title is 9.991939463727586
max lenght of question_body is 161.18703734166803
max lenght of question_answer is 163.19608488238197


In [77]:
qt_max = np.mean([len(simple_prepro(l)) for l in list(train["question_title"].values)])
qb_max = np.mean([len(simple_prepro(l))  for l in list(train["question_body"].values)])
an_max = np.mean([len(simple_prepro(l))  for l in list(train["answer"].values)])
print("mean lenght of question_title is",qt_max)
print("mean lenght of question_body is",qb_max)
print("mean lenght of question_answer is",an_max)

mean lenght of question_title is 9.991939463727586
mean lenght of question_body is 161.18703734166803
mean lenght of question_answer is 163.19608488238197


In [149]:
max_word = 35000
max_len = 1024
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_word)

In [150]:
for i in ["question_title", "question_body", "answer"]:
    tokenizer.fit_on_texts(train[i])    
print(len(tokenizer.word_index))

54531


In [151]:
sequences_title = tokenizer.texts_to_sequences(train["question_title"])
sequences_body = tokenizer.texts_to_sequences(train["question_body"])
sequences_answer = tokenizer.texts_to_sequences(train["answer"])

for i in [sequences_title,sequences_body,sequences_answer]:
    print(len(i[0]))
    print(i[0])

13
[38, 71, 4, 3392, 46, 54, 1340, 5406, 281, 7, 3, 2131, 849]
146
[119, 877, 304, 16, 2131, 2987, 15, 1, 2148, 233, 6744, 849, 5154, 849, 4403, 15, 3, 1684, 849, 2537, 1340, 5406, 4, 41, 47, 2, 49, 876, 16, 12, 1, 656, 16, 1, 2124, 4, 117, 5, 9, 1302, 5, 1552, 6, 2837, 487, 5, 4704, 34, 263, 12, 1130, 29, 784, 2, 168, 5314, 233, 1993, 10684, 115, 22, 2191, 5, 6062, 4, 63, 2, 17, 227, 2, 3285, 887, 10684, 4, 839, 9, 13, 12, 10558, 6, 25582, 2837, 37, 17, 7, 597, 174, 33, 44, 1205, 26, 1518, 303, 5, 3, 2131, 849, 195, 6253, 15566, 2131, 165, 4, 71, 23, 193, 1172, 8, 538, 164, 1074, 849, 32, 1287, 5, 1, 2453, 1340, 5406, 1027, 13, 1292, 5852, 833, 38, 71, 4, 3392, 46, 54, 5406, 6482, 16, 3, 443, 849, 195, 25583, 985, 31, 214, 281, 7, 3, 2131, 849]
151
[4, 56, 394, 1340, 5406, 33, 715, 1, 23081, 38, 71, 4, 3392, 46, 54, 5406, 3, 136, 8697, 653, 7, 580, 2235, 9, 833, 30, 1, 108, 7, 1, 849, 2, 1, 1305, 25, 1327, 35, 580, 547, 2257, 2439, 16, 1, 492, 9, 606, 437, 3285, 2484, 250, 941, 2, 69,

In [152]:
title_pad = pad_sequences(sequences_title, maxlen=max_len)
body_pad =  pad_sequences(sequences_body, maxlen=max_len)
answer_pad =  pad_sequences(sequences_answer, maxlen=max_len)

In [153]:
train["category"].value_counts()

TECHNOLOGY       2441
STACKOVERFLOW    1253
CULTURE           963
SCIENCE           713
LIFE_ARTS         709
Name: category, dtype: int64

In [154]:
cate.shape

(6079, 5)

In [155]:
type2int = {type:i for i,type in enumerate(list(set(train["category"])))}
cate = np.identity(5)[np.array(train["category"].apply(lambda x:type2int[x]))].astype(np.float64)
cate_test = np.identity(5)[np.array(test["category"].apply(lambda x:type2int[x]))].astype(np.float64)

In [156]:
cate

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [157]:
catego = np.zeros(title_pad.shape)
catego[:cate.shape[0],:cate.shape[1]] = cate

In [158]:
train_x = np.array([title_pad, body_pad, answer_pad,catego])

In [159]:
train_x.shape

(4, 6079, 1024)

In [196]:
from keras.layers import LSTM, Dropout, Input, Conv1D
from keras import backend


In [197]:
backend.clear_session()

In [192]:
model = Sequential()
model.add(Embedding(max_word,4, input_length=max_len))
model.add(LSTM(512))
model.add(Dense(128, activation='relu')) # output = 1
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.compile(optimizer='adam', loss='binary_crossentropy')

In [198]:
def define_model(length, vocab_size):
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 100)(inputs1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 100)(inputs3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='multichannel.png')
	return model

In [199]:
model = define_model(1024, len(tokenizer.word_index))

NameError: name 'Conv1D' is not defined

In [193]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1024, 4)           140000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1058816   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
Total params: 1,272,736
Trainable params: 1,272,736
Non-trainable params: 0
_________________________________________________________________


In [181]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

hist = model.fit(train_x, train_y, epochs = 300, validation_split=0.2, callbacks = [es])

ValueError: Error when checking input: expected embedding_1_input to have 2 dimensions, but got array with shape (4, 6079, 1024)

In [31]:
num_folds = 10
fold_scores = []
kf = KFold(n_splits = num_folds, shuffle = True, random_state = 42)
test_preds = np.zeros((len(test_features), len(target_cols)))
valid_preds = np.zeros((train_features.shape[0],30))
for train_index, val_index in kf.split(train_features):
    gc.collect()
    train_X = train_features[train_index, :]
    train_y = train[target_cols].iloc[train_index]
    
    val_X = train_features[val_index, :]
    val_y = train[target_cols].iloc[val_index]
    
    model = Sequential()
    model.add(LSTM(1024, input_shape=(train_features.shape[1],), activation='relu'))
    model.add(Dense(512, activation='relu')) # output = 1
    model.add(Dense(len(target_cols), activation='sigmoid')) 
    
    es = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy')
    
    model.fit(train_X, train_y, epochs = 300, validation_data=(val_X, val_y), callbacks = [es])
    preds = model.predict(val_X)
    valid_preds[val_index] = preds
    overall_score = 0
    for col_index, col in enumerate(target_cols):
        overall_score += spearmanr(preds[:, col_index], val_y[col].values).correlation/len(target_cols)
        print(col, spearmanr(preds[:, col_index], val_y[col].values).correlation)
    fold_scores.append(overall_score)
    print(overall_score)
    test_preds += model.predict(test_features)/num_folds
print(fold_scores)

ValueError: Error when checking input: expected lstm_1_input to have 3 dimensions, but got array with shape (5471, 503)

In [ ]:
valid = 0
for col_index, col in enumerate(target_cols):
    valid += spearmanr(valid_preds[:, col_index], train[col].values).correlation/30
print("valid score is ",valid)

In [ ]:
sub = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")
for col_index, col in enumerate(target_cols):
    sub[col] = test_preds[:, col_index]
sub.to_csv("submission.csv", index = False)

The contribution of the score was not great, but if you use bert etc. instead of brown, I think the score will go up more.